# Classificazione Binaria: `selfMade` con rulebased RIPPER

In [1]:
import pandas as pd
import wittgenstein as lw
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score,confusion_matrix

X_train = pd.read_csv("../data/splitted/X_train.csv")
X_test = pd.read_csv("../data/splitted/X_test.csv")
y_train = pd.read_csv("../data/splitted/y_train.csv").values.ravel()
y_test = pd.read_csv("../data/splitted/y_test.csv").values.ravel()

k=2
model = lw.RIPPER(k=k)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print("TRAIN METRICS")
print(classification_report(y_train, y_pred_train))
train_cm = confusion_matrix(y_train, y_pred_train)
print(train_cm)
print("\nTEST METRICS")
print(classification_report(y_test, y_pred_test))
test_cm = confusion_matrix(y_test, y_pred_test)
print(test_cm)

print("\nRegole generate:")
print(model.ruleset_)


TRAIN METRICS
              precision    recall  f1-score   support

       False       0.43      0.97      0.60       640
        True       0.97      0.44      0.60      1441

    accuracy                           0.60      2081
   macro avg       0.70      0.70      0.60      2081
weighted avg       0.80      0.60      0.60      2081

[[620  20]
 [812 629]]

TEST METRICS
              precision    recall  f1-score   support

       False       0.43      0.95      0.59       160
        True       0.95      0.43      0.60       361

    accuracy                           0.59       521
   macro avg       0.69      0.69      0.59       521
weighted avg       0.79      0.59      0.59       521

[[152   8]
 [204 157]]

Regole generate:
[[population_country=>1366417754.0] V [population_country=83429615.0-328239523.0^category_Technology=True] V [population_country=83429615.0-328239523.0^category_Food&Beverage=False^gross_tertiary_education_enrollment=65.6-81.9] V [category_Diversified=Fa

In [2]:
import os 

train_metrics = {
    "Accuracy": accuracy_score(y_train, y_pred_train),
    "Precision": precision_score(y_train, y_pred_train),
    "Recall": recall_score(y_train, y_pred_train),
    "F1-Score": f1_score(y_train, y_pred_train)
}
test_metrics = {
    "Accuracy": accuracy_score(y_test, y_pred_test),
    "Precision": precision_score(y_test, y_pred_test),
    "Recall": recall_score(y_test, y_pred_test),
    "F1-Score": f1_score(y_test, y_pred_test)
}

output_dir = "../results/classification_selfMade/rule_based/direct"
os.makedirs(output_dir, exist_ok=True)

filename = os.path.join(output_dir, "ripper_rule_based.txt")
with open(filename, "w") as f:
    f.write("Modello: RIPPER (Rule-Based) con livello di pruning "+str(k)+ "\n")
    f.write("\nTRAIN METRICS:\n")
    for k, v in train_metrics.items():
        f.write(f"{k}: {v:.4f}\n")
    f.write("Confusion Matrix:\n")
    f.write(f"{train_cm}\n\n")    

    f.write("\nTEST METRICS:\n")
    for k, v in test_metrics.items():
        f.write(f"{k}: {v:.4f}\n")
    f.write("Confusion Matrix:\n")
    f.write(f"{test_cm}\n")    

    f.write("\nREGOLE APPRESE:\n")
    f.write(str(model.ruleset_))

print(f"✔ Metriche salvate in: {filename}")

✔ Metriche salvate in: ../results/classification_selfMade/rule_based/direct\ripper_rule_based.txt
